In [1]:
import os
import keras
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn-white")
import os

C:\Users\gimon\AppData\Local\Temp\ipykernel_5172\2844206087.py:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-white")


In [2]:
img_train_male = "dataset/Training/male"
img_train_female = "dataset/Training/female"
img_valid_male = "dataset/Validation/male"
img_valid_female = "dataset/Validation/female"
size = 64

In [3]:
def preprocess_image(img_path):
    img = keras.utils.load_img(img_path, target_size=(size,size))
    tensor = keras.utils.img_to_array(img)
    tensor /= 255.0 
    return tensor

In [4]:
categories = ["male", "female"]
nb_class = len(categories)
x = []
y = []

# variables to test the face with a mask
test_on_male_x = []
test_on_male_y = []

# variables to test the face with a mask
test_female_x = []
test_female_y = []



In [5]:
#a=0
# preprocess the images, each of which is a face with mask
for i in os.listdir(img_train_male):
    img_path = os.path.join(img_train_male, i)
    img_tensor = preprocess_image(img_path)
    x.append(img_tensor)
    y.append(0)
 #   if a < 10:
 #       print ("len\[x\]:  %d" % len(x) )
 #   a=a+1

In [6]:
# preprocess the images, each of which is a face without mask
for i in os.listdir(img_train_female):
    img_path = os.path.join(img_train_female, i)
    img_tensor = preprocess_image(img_path)
    x.append(img_tensor)
    y.append(1)


In [7]:
x = np.array(x)
y = np.array(y)
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.1)
Y_train = keras.utils.to_categorical(Y_train, 2)
Y_test = keras.utils.to_categorical(Y_test, 2)

In [15]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, BatchNormalization, Dropout
from tensorflow.keras import regularizers

model = keras.models.Sequential()
# Conv2D
# the # of convolution filters 
# the number of rows and columns in terms of a convolution kernel
# padding
#...
model.add(Conv2D(32, kernel_size = (3, 3), padding = "same", activation = "relu", input_shape=(size, size, 3)))
################## start - add custum layers
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (3, 3), strides = (2, 2)))

model.add(Conv2D(96, kernel_size = (5, 5), strides = (1, 1), padding = "same", kernel_regularizer=regularizers.l2(l2 = 1e-4), activation = "relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=regularizers.l2(l2 = 1e-4), activation = "relu"))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=regularizers.l2(l2 = 1e-4), activation = "relu"))
model.add(BatchNormalization())

model.add(Conv2D(96, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=regularizers.l2(l2 = 1e-4), activation = "relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

################## end - add custum layers

model.add(Flatten())
model.add(Dense(1024, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.5))

model.add(Dense(2, activation = "softmax"))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 batch_normalization_5 (Bat  (None, 64, 64, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 31, 31, 96)        76896     
                                                                 
 batch_normalization_6 (Bat  (None, 31, 31, 96)        384       
 chNormalization)                                                
                                                      

In [9]:
from keras.utils import plot_model
plot_model(model, show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [11]:
from tensorflow.keras.utils import to_categorical, Sequence
import cv2, sklearn

def zero_one_scaler(image):
    return image/255.0

def get_preprocessed_ohe(images, labels, pre_func=None):
    # preprocessing 함수가 입력되면 이를 이용하여 image array를 scaling 적용.
    if pre_func is not None:
        images = pre_func(images)
    # OHE 적용    
    oh_labels = to_categorical(labels)
    return images, oh_labels

IMAGE_SIZE = 128
BATCH_SIZE = 64

class Dataset(Sequence):
    def __init__(self, images_array, labels, batch_size = BATCH_SIZE, augmentor = None, shuffle = False, pre_func = None):
        self.images_array = images_array
        self.labels = labels
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.pre_func = zero_one_scaler
        self.shuffle = shuffle
        
    def __len__(self):
        return int(np.ceil(len(self.labels) / self.batch_size))
    
    def __getitem__(self, index):
        images_fetch = self.images_array[index * self.batch_size:(index+1) * self.batch_size]
        if self.labels is not None:
            label_batch = self.labels[index * self.batch_size:(index+1) * self.batch_size]
        image_batch = np.zeros((images_fetch.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3), dtype='float32')
        
        # batch_size에 담긴 건수만큼 iteration 하면서 opencv image load -> image augmentation 변환(augmentor가 not None일 경우)-> image_batch에 담음. 
        for image_index in range(images_fetch.shape[0]):
            #image = cv2.cvtColor(cv2.imread(image_name_batch[image_index]), cv2.COLOR_BGR2RGB)
            # 원본 image를 IMAGE_SIZE x IMAGE_SIZE 크기로 변환
            image = cv2.resize(images_fetch[image_index], (IMAGE_SIZE, IMAGE_SIZE))
            # 만약 augmentor가 주어졌다면 이를 적용. 
            if self.augmentor is not None:
                image = self.augmentor(image=image)['image']
                
            # 만약 scaling 함수가 입력되었다면 이를 적용하여 scaling 수행. 
            if self.pre_func is not None:
                image = self.pre_func(image)
            
            # image_batch에 순차적으로 변환된 image를 담음.               
            image_batch[image_index] = image
        
        return image_batch, label_batch
    
    def on_epoch_end(self):
        if self.shuffle: self.images_array, self.labels = sklearn.utils.shuffle(self.images_array, self.labels)
    
tr_ds = Dataset(X_train, Y_train, shuffle = True)
val_ds = Dataset(X_test, Y_test)

In [16]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

rlr_cb = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 5, mode = 'min', verbose = 1)
ely_cb = EarlyStopping(monitor = 'val_loss', patience = 4, mode = 'min', verbose = 1)

model.fit(X_train, Y_train, epochs = 30, validation_split = 0.1, callbacks = [rlr_cb, ely_cb])

Epoch 1/30
  16/1190 [..............................] - ETA: 17:17 - loss: 2.7734 - accuracy: 0.5762

In [ ]:
# evaluate a deep learning model
prediction = model.predict(X_test)
loss, acc = model.evaluate(X_test, Y_test, verbose=2)

In [ ]:
 # draw images with accuracies and labels
topCnt = 8 *10
if len(X_test) < topCnt:
    topCnt = len(X_test)
    
plt.figure(figsize=(15,15))
for idx in range(topCnt):
    plt.subplot(8, 10, idx+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_test[idx], cmap=plt.cm.binary)
    
    if prediction[idx][0] > prediction[idx][1]:
        label = "Male " + str(int(prediction[idx][0] * 100)) + " %"
    else:
        label = "Female " + str(int(prediction[idx][1] * 100)) + " %"
    plt.xlabel(label)
plt.show()